In [22]:
from urllib import request
import json
import time
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta

def get_data(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.89 Safari/537.36'
    }
    req = request.Request(url,headers = headers)
    response = request.urlopen(req)
    if response.getcode() == 200:
        return response.read()
    return None

def parse_data(html):
    data = json.loads(html)['cmts']
    comments = []
    for item in data:
        comment = {
            'id':item['id'],
            'nickName':item['nickName'],
            'cityName':item['cityName'] if 'cityName' in item else'',#处理cityName不存在的情况
            'content':item['content'].replace('\n','',10), #处理评论内容的换情况
            'score':item['score'],
            'startTime':item['startTime']
        }
        print(comment)
        comments.append(comment)
    return comments


def save_to_txt():
    start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(start_time)
    end_time = '2019-10-10 00:00:00'
    df = pd.DataFrame(columns=['id','content','score'])
    while start_time > end_time:
        url = 'http://m.maoyan.com/mmdb/comments/movie/1277939.json?_v_==yes&offset=0&startTime=' + start_time.replace(' ','%20')
        html = None
        """
            问题：当请求过于频繁时，服务器会拒绝连接，实际上是服务器的反爬虫策略
            解决：1.在每个请求增加延时0.1秒，尽量减少请求别拒绝
                 2.如果被拒绝，则0.5秒后重试
        """
        try:
            html = get_data(url)
        except Exception as e:
            time.sleep(0.5)
            html = get_data(url)
        else:
            time.sleep(0.1)      
        comments = parse_data(html)            
        start_time = comments[14]['startTime']
        start_time = datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1) #转换为datetime类型，减1秒，避免获取到重复数据
        start_time = datetime.strftime(start_time,'%Y-%m-%d %H:%M:%S') #转换为str     
        for item in comments:
            item_list = []
            item_list.append(item['id'])
            item_list.append(item['content'])
            item_list.append(item['score'])
            item_list.append(item['startTime'])
            with open("comments.csv", "a", newline='',encoding="utf_8_sig") as file:
                writer = csv.writer(file,delimiter=',')
                writer.writerow(item_list)
                
if __name__ == "__main__":
    html = get_data('http://m.maoyan.com/mmdb/comments/movie/1277939.json?_v_==yes&offset=0&startTime=2019-10-08%2022%3A25%3A03')
    comments = parse_data(html)
    print(comments)
    save_to_txt()

{'id': 1087008214, 'nickName': '张晓', 'cityName': '太原', 'content': '陪同家里老人一起看，一起追忆童年的美好时光，什么时候有黑白电视的什么时候有了彩色电视机，奥运会的时候我们在哪里在干嘛？看女排的时候才是小学的时候，听着我和我的祖国看着感人的故事眼泪一次又一次的洒落，伟大的祖国我们永远爱你！感谢导演感谢演员给我们的回忆和激动！', 'score': 5, 'startTime': '2019-10-08 22:24:57'}
{'id': 1086970897, 'nickName': '曲峰', 'cityName': '济南', 'content': '近期工作生活狼狈不堪，自己网购一张票去看了祖国一片。热血澎湃，故事个个精彩！我就是国，自己更优秀中国就更优秀！加油！我和我的国！', 'score': 5, 'startTime': '2019-10-08 22:24:45'}
{'id': 1087010480, 'nickName': 'huyuxiaomao', 'cityName': '贵阳', 'content': '不行，不如机长和攀登者，完全是靠爱国热情炒的票房', 'score': 3, 'startTime': '2019-10-08 22:24:22'}
{'id': 1087006557, 'nickName': 'LL超人不会飞', 'cityName': '扬州', 'content': '题材伟大。电影除了葛优，，其余演员一般。', 'score': 3, 'startTime': '2019-10-08 22:24:19'}
{'id': 1086974742, 'nickName': '志龙女友呢', 'cityName': '承德', 'content': '情怀  祖国  很有感觉', 'score': 5, 'startTime': '2019-10-08 22:24:17'}
{'id': 1087006553, 'nickName': '\ue04a太阳 \ue04a', 'cityName': '鸡西', 'content': '感人！每个故事都纯朴感人！', 'score': 5, 'startTime': '2019-10-08 22:24:07'}


{'id': 1087586157, 'nickName': 'xmj_1010', 'cityName': '衡阳', 'content': '电影比较适合老年人看 除了香港回归有点感人 其他的都好无聊', 'score': 2.5, 'startTime': '2019-10-14 15:53:35'}
{'id': 1087601288, 'nickName': '菲家小猫', 'cityName': '南京', 'content': '为了女排，我觉得女排值得一张票，张译眼神真绝了，北京那片段get不到，可能因为不是北京人，get不到生活习惯。', 'score': 4.5, 'startTime': '2019-10-14 15:52:50'}
{'id': 1087579319, 'nickName': 'mark陈继勇', 'cityName': '上海', 'content': '可以精彩祖国万岁！', 'score': 5, 'startTime': '2019-10-14 15:51:37'}
{'id': 1087601292, 'nickName': '竹子林之宽容', 'cityName': '咸阳', 'content': '一部不错的影片', 'score': 5, 'startTime': '2019-10-14 15:51:34'}
{'id': 1087600412, 'nickName': '晓雄', 'cityName': '上海', 'content': '一般有点失望', 'score': 1, 'startTime': '2019-10-14 15:51:20'}
{'id': 1087603302, 'nickName': 'tigersantana', 'cityName': '北京', 'content': '不错，很好啊', 'score': 5, 'startTime': '2019-10-14 15:51:13'}
{'id': 1087586147, 'nickName': 'hellojane2007', 'cityName': '佛山', 'content': '带爸妈过来看，很好', 'score': 5, 'startTime': '2019-10-14 15:50:49'}
{'id': 1087

{'id': 1087604006, 'nickName': '蛋蛋君', 'cityName': '西宁', 'content': '估计是不在国庆期间看的电影，没有想象中那么强烈的感情，但依然敬佩英雄的精神！向他们致敬！', 'score': 5, 'startTime': '2019-10-14 15:16:12'}
{'id': 1087597944, 'nickName': 'wwydengguan', 'cityName': '克拉玛依', 'content': '恢宏大气的电影，每一个故事对新中国来说都具有里程碑的意义，我们的祖国越来越强大，离不开全中国人民的努力奋斗！很受鼓舞和感动！希望以后多一些这样的电影！', 'score': 5, 'startTime': '2019-10-14 15:15:15'}
{'id': 1087597941, 'nickName': 'ad_门徒', 'cityName': '天津', 'content': '挺不错的，应该多进行一下爱国教育', 'score': 4.5, 'startTime': '2019-10-14 15:14:55'}
{'id': 1087581747, 'nickName': 'cRG719004762', 'cityName': '济南', 'content': '祖国万岁！！！', 'score': 5, 'startTime': '2019-10-14 15:14:53'}
{'id': 1087595970, 'nickName': '妮子', 'cityName': '石狮', 'content': '前面好看，后面就不怎么好看了', 'score': 3, 'startTime': '2019-10-14 15:14:31'}
{'id': 1087600103, 'nickName': '周琴', 'cityName': '重庆', 'content': '感动好看热泪盈', 'score': 5, 'startTime': '2019-10-14 15:12:16'}
{'id': 1087597915, 'nickName': 'xfh813195469', 'cityName': '沈阳', 'content': '好看(｡･ω･｡)ﾉ♡', 'score': 5,

{'id': 1087595617, 'nickName': '霜', 'cityName': '乐清', 'content': '值得大人和孩子们一起观看的一部电影，让我们知道我们的祖国人民的团结一致，无私奉献精神。', 'score': 5, 'startTime': '2019-10-14 14:34:36'}
{'id': 1087592832, 'nickName': '憬安', 'cityName': '惠阳', 'content': '真的很好看，抑制不住的流泪', 'score': 5, 'startTime': '2019-10-14 14:34:21'}
{'id': 1087594720, 'nickName': 'srt707113727', 'cityName': '济南', 'content': '都是历史性的时刻，感动', 'score': 5, 'startTime': '2019-10-14 14:33:45'}
{'id': 1087597549, 'nickName': 'Seal三只松鼠', 'cityName': '河源', 'content': '非常好看值得', 'score': 5, 'startTime': '2019-10-14 14:32:01'}
{'id': 1087580703, 'nickName': 'naturewind19', 'cityName': '长沙', 'content': '白昼流星是败笔', 'score': 4.5, 'startTime': '2019-10-14 14:31:47'}
{'id': 1087594705, 'nickName': 'BNU366717154', 'cityName': '合肥', 'content': '还可以吧，有几个故事挺喜欢的', 'score': 3.5, 'startTime': '2019-10-14 14:31:41'}
{'id': 1087597538, 'nickName': '叫我嗷嗷宝宝', 'cityName': '洛阳', 'content': '好看好看好看', 'score': 5, 'startTime': '2019-10-14 14:30:40'}
{'id': 1087580701, 'nickName': 

{'id': 1087489747, 'nickName': '洋葱炝炒沙丁鱼', 'cityName': '杭州', 'content': '个人偏喜欢天安门升国旗篇章和夺冠篇章  影片整体很震撼  身为中国人的自豪感油然而生。夺冠篇章里的小孩演的很好  幽默又不偏离为祖国献礼的主题  于细微处流露情感。赞赞', 'score': 5, 'startTime': '2019-10-14 13:59:55'}
{'id': 1087597284, 'nickName': '张闲女.', 'cityName': '东营', 'content': '太长了吧欧豪太好看了', 'score': 5, 'startTime': '2019-10-14 13:59:35'}
{'id': 1087594452, 'nickName': '浅笑无痕0118', 'cityName': '绍兴', 'content': '主要是在国庆档播放这部电影非常合时宜，卖的是情怀，还是可以的。', 'score': 4, 'startTime': '2019-10-14 13:59:18'}
{'id': 1087592594, 'nickName': '你不找，我不扰。', 'cityName': '西安', 'content': '很好很强大！', 'score': 5, 'startTime': '2019-10-14 13:58:30'}
{'id': 1087592588, 'nickName': '筱珊543', 'cityName': '巨野', 'content': '国之强大    国之光荣  ', 'score': 5, 'startTime': '2019-10-14 13:58:10'}
{'id': 1087557888, 'nickName': '张小娥', 'cityName': '深圳', 'content': '好看好看，感动了！', 'score': 5, 'startTime': '2019-10-14 13:58:00'}
{'id': 1087580558, 'nickName': 'Aomilk.xia', 'cityName': '长沙', 'content': '我爱种花家❤️❤️❤️', 'score': 4.5, 'startTime':

...

{'id': 1087138202, 'nickName': '食从骑士手中来', 'cityName': '武汉', 'content': '相遇和北京你好好看', 'score': 3.5, 'startTime': '2019-10-10 08:10:44'}
{'id': 1087138198, 'nickName': '哈哈哈嘿嘿嘿嚯嚯', 'cityName': '海口', 'content': '强行煽情的地方太多，总体还是不错的', 'score': 4.5, 'startTime': '2019-10-10 08:10:06'}
{'id': 1087137203, 'nickName': '董君🎶', 'cityName': '哈尔滨', 'content': '一刻也不能分割，', 'score': 5, 'startTime': '2019-10-10 08:10:01'}
{'id': 1087136006, 'nickName': 'YGX896997795', 'cityName': '南昌', 'content': '很好看很激动很感动很骄傲', 'score': 5, 'startTime': '2019-10-10 08:09:49'}
{'id': 1087136002, 'nickName': '小飞龙', 'cityName': '中牟', 'content': '看到激动人心的时刻，都泪流满面', 'score': 5, 'startTime': '2019-10-10 08:09:25'}
{'id': 1087137202, 'nickName': '烟雨芳霏', 'cityName': '淄博', 'content': '一部可以震撼国人心灵深处的影片，我的大中国 永远为你感到自豪！', 'score': 5, 'startTime': '2019-10-10 08:09:24'}
{'id': 1087126999, 'nickName': '十一.', 'cityName': '舟山', 'content': '好看好看好看', 'score': 5, 'startTime': '2019-10-10 08:08:56'}
{'id': 1087117813, 'nickName': '芷晴宝宝520', 'ci

{'id': 1087137016, 'nickName': 'jXp282527706', 'cityName': '抚顺', 'content': '非常好看，真不错', 'score': 4, 'startTime': '2019-10-10 07:26:22'}
{'id': 1087130875, 'nickName': 'mpZ856164330', 'cityName': '郯城', 'content': '和家人一起看的，特别好看，感人。个人很喜欢前夜，夺冠，回归，奥运……超出预期的好看', 'score': 5, 'startTime': '2019-10-10 07:26:02'}
{'id': 1086109535, 'nickName': '和平', 'cityName': '余姚', 'content': '细节处理的比较好！', 'score': 5, 'startTime': '2019-10-10 07:24:53'}
{'id': 1087108444, 'nickName': '宁静＠_＠致远', 'cityName': '忠县', 'content': '很不错，按照时间顺序从建国到现在的大事件都记录了', 'score': 4.5, 'startTime': '2019-10-10 07:23:38'}
{'id': 1087138023, 'nickName': 'mmwx99', 'cityName': '南京', 'content': '真的很好看。有血有肉。', 'score': 5, 'startTime': '2019-10-10 07:21:18'}
{'id': 1087137001, 'nickName': 'ZHBN', 'cityName': '贵阳', 'content': '好看，很不错', 'score': 5, 'startTime': '2019-10-10 07:21:06'}
{'id': 1087138018, 'nickName': '七月', 'cityName': '连云港', 'content': '感谢#小丸子观影团#每个片段都很让人为之动容。一开始开国大典，每一家都来排队捐赠，很感动。五星红旗迎风飘扬，庆幸自己是中国人。', 'score': 4.5, 'startTime':

{'id': 1087113725, 'nickName': '。。。', 'cityName': '长沙', 'content': '我觉得挺好的', 'score': 5, 'startTime': '2019-10-10 03:19:26'}
{'id': 1087130579, 'nickName': 'PXH953593319', 'cityName': '保定', 'content': '心情一直都很激荡，在这样一个时间背景下的，这样好的一个片子……想不火……难', 'score': 5, 'startTime': '2019-10-10 03:13:35'}
{'id': 1087127730, 'nickName': '做人应该像我一样', 'cityName': '西安', 'content': '时间太长了，不过故事恰到好处。每个故事都很精彩。剧组用心了。', 'score': 5, 'startTime': '2019-10-10 03:11:39'}
{'id': 1087131044, 'nickName': 'dhY520324123', 'cityName': '曲靖', 'content': '电影也算好看', 'score': 4, 'startTime': '2019-10-10 03:11:17'}
{'id': 1087127713, 'nickName': 'Pbn997626881', 'cityName': '萍乡', 'content': '太喜欢杜江了，', 'score': 5, 'startTime': '2019-10-10 02:55:40'}
{'id': 1087128671, 'nickName': '大站集团有限公司', 'cityName': '郑州', 'content': '我爱我的祖国！', 'score': 5, 'startTime': '2019-10-10 02:39:31'}
{'id': 1087113700, 'nickName': '奈何缘浅', 'cityName': '济南', 'content': '影片中体现了各个基层的爱国情怀，为我们充分展示对于国家荣誉面前，牺牲小我，成就大我的精神，没当国歌想起我的心情异常澎湃，我相信每个中国人都深有体会，我们中国一点点的崛起，逐步

{'id': 1087130183, 'nickName': 'rmQ978331401', 'cityName': '永康', 'content': '超级好看，，，', 'score': 5, 'startTime': '2019-10-10 00:42:10'}
{'id': 1087128325, 'nickName': '梦回归处', 'cityName': '邢台', 'content': '特别特别特别好', 'score': 5, 'startTime': '2019-10-10 00:42:07'}
{'id': 1087123756, 'nickName': '安若少年。', 'cityName': '合肥', 'content': '此生无悔入华夏，来世还为中国魂。', 'score': 5, 'startTime': '2019-10-10 00:40:35'}
{'id': 1084337072, 'nickName': 'azhong0831', 'cityName': '北京', 'content': '太好看了，那些平凡的人和不平凡的事让我们哭着笑着走到了祖国的七十华诞，我爱我的祖国', 'score': 5, 'startTime': '2019-10-10 00:39:46'}
{'id': 1087128307, 'nickName': 'VuA962067082', 'cityName': '泸州', 'content': '看的是爱国情怀', 'score': 5, 'startTime': '2019-10-10 00:38:59'}
{'id': 1087113555, 'nickName': '专骂杠精', 'cityName': '佛山', 'content': '由不同的故事组成，从1949年到2019年期间的故事，也是我们伟大祖国的变化，从复兴到繁荣之路，也有1997年7月1日香港回归，2008北京奥运会等，70年了，我们的国家越来越好，非常正能量的一部电影', 'score': 5, 'startTime': '2019-10-10 00:37:21'}
{'id': 1087128297, 'nickName': '朱天悦123', 'cityName': '上海', 'content': '很棒的电影啊。'

{'id': 1087123546, 'nickName': 'VmW787476199', 'cityName': '成都', 'content': '祖国生日快乐', 'score': 5, 'startTime': '2019-10-10 00:01:56'}
{'id': 1087127102, 'nickName': 'xVS892694541', 'cityName': '深圳', 'content': '还行，国外华人和游子看了应该会有更深刻的感想。', 'score': 4, 'startTime': '2019-10-10 00:01:39'}
{'id': 1087123541, 'nickName': 'IXD805394752', 'cityName': '西安', 'content': '感谢#小丸子观影团#总有一群人，为了国家、人民的安全而奋战在前线；总有一群人，为了国家、人民的荣耀而奋不顾身；也总有那么一群人，为了国家、人民的强大而不惜隐姓埋名——潜心科研。那有什麽歲月靜好，只是有人為我們負重前行。\ue41d祝願我們偉大的祖國繁榮富強！！！', 'score': 5, 'startTime': '2019-10-10 00:01:38'}
{'id': 1087123542, 'nickName': 'bmz371297375', 'cityName': '防城港', 'content': '值得一看好电影', 'score': 5, 'startTime': '2019-10-10 00:01:34'}
{'id': 1087113446, 'nickName': 'ゞｙanゞ', 'cityName': '台山', 'content': '感谢#小丸子观影团#，感动，祖国强大，人民幸福，感谢不能忘了对祖国做出贡献和牺牲了的每一个人，没有你们就没有今天强大的中国！好看的很，把祖国的每个重要时刻都感受一次，非常震撼感到无比的自豪', 'score': 5, 'startTime': '2019-10-10 00:00:59'}
{'id': 1087128076, 'nickName': 'Grace-关绮文', 'cityName': '珠海', 'content': '燃爆的电影每个小故事都很有代表性 感动', 'score': 